In [1]:
# imports 
import pandas as pd 
import requests
import tqdm as tqdm
tqdm.tqdm.pandas()

In [2]:
from sympy import use


df = pd.read_csv("/Users/kaleemullahqasim/Documents/GitHub/AdIdentifer_Downloader/v0.2/1-12600_5515_ad_websites.csv", usecols=['url'])

In [3]:
df

,url
0,youtube.com
1,twitter.com
2,doubleclick.net
3,fastly.net
4,bing.com
...,...
5510,filmdaily.co/obsessions/true-crime/mary-vincen...
5511,www.mirror.co.uk/news/uk-news/lee-rigbys-mum-n...
5512,etherealgames.com
5513,www5.himovies.to/watch-tv/the-nurse-96013.9585466


In [4]:
# add https:// to the url if it is not present
def add_https(url):
    if 'http' not in url:
        return 'https://' + url
    return url

df['url'] = df['url'].progress_apply(add_https)

100%|██████████| 5515/5515 [00:00<00:00, 2365918.64it/s]


In [5]:
# # send request to the url and get the status code if the status code is 200 then the website is up and running
# def get_status_code(url):
#     try:
#         r = requests.get(url)
#         return r.status_code
#     except:
#         return 'error'
    
# df['status_code'] = df['url'].progress_apply(get_status_code)

In [6]:
import concurrent.futures
import requests
from tqdm import tqdm

def get_status_code(url):
    try:
        r = requests.get(url, timeout=5)
        return r.status_code
    except:
        return 'error'

def main():
    df['status_code'] = ''
    urls = df['url'].tolist()
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(get_status_code, url): url for url in urls}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(urls)):
            url = futures[future]
            try:
                data = future.result(timeout=5)
            except concurrent.futures.TimeoutError:
                print(f'Timeout for {url}')
            except Exception as e:
                print(f'Error for {url}: {e}')
            else:
                df.loc[df['url'] == url, 'status_code'] = data

if __name__ == "__main__":
    main()

100%|██████████| 5515/5515 [10:16<00:00,  8.94it/s]


In [7]:
df

,url,status_code
0,https://youtube.com,200
1,https://twitter.com,400
2,https://doubleclick.net,200
3,https://fastly.net,200
4,https://bing.com,200
...,...,...
5510,https://filmdaily.co/obsessions/true-crime/mar...,error
5511,https://www.mirror.co.uk/news/uk-news/lee-rigb...,200
5512,https://etherealgames.com,403
5513,https://www5.himovies.to/watch-tv/the-nurse-96...,error


In [8]:
df.to_csv("/Users/kaleemullahqasim/Documents/GitHub/AdIdentifer_Downloader/v0.2/200_of_all_URLs.csv", index=False)